# Assignment 2: Voting Visualized

## Deadline

Oct. 24th

## Important notes

- Make sure you push on GitHub your notebook with all the cells already evaluated.
- Note that maps do not render in a standard Github environment. You should export them to HTML and link them in your notebook.
- Don't forget to add a textual description of your thought process, the assumptions you made, and the solution you implemented.
- Please write all your comments in English, and use meaningful variable names in your code.

## Background


* Are you curious to know what the political leanings of the people of Switzerland are?
* Do you wake up in a cold sweat, wondering which party won the last cantonal parliament election in Vaud?
* Are you looking to learn all sorts of visualizations, including maps, in Python?

If your answer to any of the above is yes, this assignment is just right for you. Otherwise, it's still an assignment, so we're terribly sorry.

The chief aim of this assignment is to familiarize you with visualizations in Python, particularly maps, and also to give you some insight into how visualizations are to be interpreted. The data we will use is the data on Swiss cantonal parliament elections from 2007 to 2018, which contains, for each cantonal election in this time period, the voting percentages for each party and canton.

For the visualization part, install [Folium](Folium) (_Hint: it is not available in your standard Anaconda environment, therefore search on the Web how to install it easily!_). Folium's README comes with very clear examples, and links to their own iPython Notebooks -- make good use of this information. For your own convenience, in this same directory you can already find one TopoJSON file, containing the geo-coordinates of the cantonal borders of Switzerland.

One last, general reminder: back up any hypotheses and claims with data, since this is an important aspect of the course.

In [1]:
# Put your imports here.
import pandas as pd
import json
import numpy as np
import folium

In [2]:
data_folder = './data/'

## Task 1: Cartography and census

__A)__ Display a Swiss map that has cantonal borders as well as the national borders. We provide a TopoJSON `data/ch-cantons.topojson.json` that contains the borders of the cantons.

__B)__ Take the spreadsheet `data/communes_pop.xls`, collected from [admin.ch](https://www.bfs.admin.ch/bfs/fr/home/statistiques/catalogues-banques-donnees/tableaux.assetdetail.5886191.html), containing population figures for every commune. You can use [pd.read_excel()](https://pandas.pydata.org/pandas-docs/version/0.20/generated/pandas.read_excel.html) to read the file and to select specific sheets. Plot a histogram of the population counts and explain your observations. Do not use a log-scale plot for now. What does this histogram tell you about urban and rural communes in Switzerland? Are there any clear outliers on either side, and if so, which communes?

__C)__ The figure below represents 4 types of histogram. At this stage, our distribution should look like Fig.(a). A common way to represent [power-laws](https://en.wikipedia.org/wiki/Power_law) is to use a histogram using a log-log scale  -- remember: the x-axis of an histogram is segmented in bins of equal sizes and y-values are the average of each bin. As shown in Fig.(b), small bins sizes might introduce artifacts. Fig.(b) and Fig.(c) are examples of histograms with two different bin sizes. Another great way to visualize such distribution is to use a cumulative representation, as show in Fig.(d), in which the y-axis represents the number of data points with values greater than y.  
  
Create the figures (b) and (d) using the data extracted for task 1B. For Fig.(b), represent two histograms using two different bin sizes and provide a brief description of the results. What does this tell you about the relationship between the two variables, namely the frequency of each bin and the value (i.e. population in case of the communal data) for each bin?

<img src="plaw_crop.png" style="width: 600px;">
  
The figure is extracted from [this paper](https://arxiv.org/pdf/cond-mat/0412004.pdf) that contains more information about this family of distributions.

In [ ]:
ch_coord = [46.8, 8.3]
geo_json_data = json.load(open(data_folder +'/ch-cantons.topojson.json'))
# Stamen Terrain, Stamen Toner, Mapbox Bright, and Mapbox Control Room, cartodbpositron
m_switzerland = folium.Map(ch_coord, tiles='Mapbox Bright', zoom_start=8)

m_switzerland

In [ ]:
folium.TopoJson(
    geo_json_data, 
    object_path='objects.cantons',
    style_function=lambda feature: {
        'color' : 'blue',
        'weight' : 2
        }
).add_to(m_switzerland)

m_switzerland

In [ ]:
population = pd.read_excel(data_folder + '/communes_pop.xls', sheet_name=0, header=list(range(0, 6)))
# population.describe()

In [ ]:
population.head(20)

In [ ]:
population.tail(20)

## Task 2: Parties visualized

We provide a spreadsheet, `data/voters.xls`, (again) collected from [admin.ch](https://www.bfs.admin.ch/bfs/fr/home/statistiques/politique/elections/conseil-national/force-partis.assetdetail.217195.html), which contains the percentage of voters for each party and for each canton. For the following task, we will focus on the period 2014-2018 (the first page of the spreadsheet). Please report any assumptions you make regarding outliers, missing values, etc. Notice that data is missing for two cantons, namely Appenzell Ausserrhoden and Graubünden, and your visualisations should include data for every other canton.


__A)__ For the period 2014-2018 and for each canton, visualize, on the map, **the percentage of voters** in that canton who voted for the party [`UDC`](https://en.wikipedia.org/wiki/Swiss_People%27s_Party) (Union démocratique du centre). Does this party seem to be more popular in the German-speaking part, the French-speaking part, or the Italian-speaking part?

__B)__ For the same period, now visualize **the number of residents** in each canton who voted for UDC.

__C)__ Which one of the two visualizations above would be more informative in case of a national election with majority voting (i.e. when a party needs to have the largest number of citizens voting for it among all parties)? Which one is more informative for the cantonal parliament elections?

For part B, you can use the `data/national_council_elections.xslx` file ([guess where we got it from](https://www.bfs.admin.ch/bfs/fr/home/statistiques/politique/elections/conseil-national/participation.assetdetail.81625.html)) to have the voting-eligible population of each canton in 2015.

In [ ]:
voters = pd.read_excel(
    data_folder + '/voters.xls', 
    sheet_name=1, 
    skipfooter=55,
    header=2)

# drop columns
voters.dropna(axis=1, inplace=True, how='all')
# drop rows
voters.dropna(axis=0, inplace=True, how='all')

In [ ]:
voters.head(10)

In [ ]:
voters.tail(10)

## Task 3: More socialism or more nationalism?

In this section, we focus on two parties that are representative of the left and the right on the Swiss political spectrum. You will propose a way to visualize their influence over time and for each canton.

__A)__ Take the two parties [`UDC`](https://en.wikipedia.org/wiki/Swiss_People%27s_Party) (Union démocratique du centre) and [`PS`](https://en.wikipedia.org/wiki/Social_Democratic_Party_of_Switzerland) (Parti socialiste suisse). For each canton, we define 'right lean' in a certain period as follows:

$$\frac{VoteShare_{UDC} - VoteShare_{PS}}{VoteShare_{UDC} + VoteShare_{PS}}$$  

Visualize the right lean of each canton on the map. What conclusions can you draw this time? Can you observe the [röstigraben](https://en.wikipedia.org/wiki/R%C3%B6stigraben) ?

__B)__ For each party, devise a way to visualize the difference between its 2014-2018 vote share (i.e. percentage) and its 2010-2013 vote share for each canton. Propose a way to visualize this evolution of the party over time, and justify your choices. There's no single correct answer, but you must reasonably explain your choices.

#### 3A 

Create 26 tables, one for each cantons. In each table we store the year as an index. For each index we save the `UDC` and `PS` columns such that we can compute the 'right-lean' for each canton for the available year. We can finally plot each year 'right-lean' ?

In [3]:
def import_voters(sheet_name=0, usecols=None):
    the_voters = pd.read_excel(
        data_folder + '/voters.xls', 
        sheet_name=sheet_name, 
        skipfooter=55, # empirically
        header=2
    )
    # drop columns
    the_voters.dropna(axis=1, inplace=True, how='all')
    # drop rows
    the_voters.dropna(axis=0, inplace=True, how='all')
    # rename unnamed column by "Canton"
    the_voters.rename(columns={"Unnamed: 0": "Canton"}, inplace=True)
    
    # Replace all occurring digits and parathesis in the strings with nothing
    the_voters.Canton = the_voters['Canton'].str.replace('\d+\)', '')
    the_voters.columns = the_voters.columns.str.replace('\s\d+\)', '')
    
    # sort according to the canton's name and reset index
    the_voters.sort_values('Canton', inplace=True)
    the_voters = the_voters.set_index('Canton')
    
    # convert year into unsig. int
    the_voters["Année électorale"] = the_voters["Année électorale"].astype(np.uint16)
    
    # select relevant columns
    if usecols:
        if all([True if col in the_voters.columns.values else False for col in usecols]):
            the_voters = the_voters[usecols]
        else:
            raise KeyError(
                'usecols do not contains columns of data exracted, incopatible column(s): {}'.format(
                    [col for col in usecols if col not in the_voters.columns.values]  
                )
            )
    return the_voters

In [5]:
cols = ['Année électorale','Participation', 'PS', 'UDC']

voters_PS_UDC = {
    '{}-{}'.format(k1, k2):None for k1,k2 in zip(
        range(2007, 2015)[::-1], 
        range(2010, 2018)[::-1]
    )
}

for i, k in enumerate(voters_PS_UDC.keys()):
    voters_PS_UDC[k] = import_voters(sheet_name=i, usecols=cols)
    print(
        'Sheet {}: nb cantons {}, nb cols {}'.format(
            k, 
            voters_PS_UDC[k].shape[0], 
            voters_PS_UDC[k].shape[1]
        )
    )
    
print('nb of sheets imported {}'.format(len(voters_PS_UDC)))

Sheet 2014-2017: nb cantons 26, nb cols 4
Sheet 2013-2016: nb cantons 26, nb cols 4
Sheet 2012-2015: nb cantons 22, nb cols 4
Sheet 2011-2014: nb cantons 23, nb cols 4
Sheet 2010-2013: nb cantons 20, nb cols 4
Sheet 2009-2012: nb cantons 21, nb cols 4
Sheet 2008-2011: nb cantons 23, nb cols 4
Sheet 2007-2010: nb cantons 26, nb cols 4
nb of sheets imported 8


In [6]:
voters_PS_UDC['2014-2017'].head()

,Année électorale,Participation,PS,UDC
Canton,,,,
Appenzell Rh. Ext.,2015,36.111260,11.475833,16.281936
Appenzell Rh. Int.,2015,36.111260,NaN,NaN
Argovie,2016,32.805153,18.918913,31.937510
Berne,2018,30.516313,22.328775,26.760869
Bâle-Campagne,2015,33.845621,21.995307,26.741611


In [ ]:
colorscale = branca.colormap.linear.YlGnBu_09.scale(0, 30)

def style_function(feature):
    employed = employed_series.get(int(feature['id'][-5:]), None)
    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': '#black' if employed is None else colorscale(employed)
    }


ch_coord = [46.8, 8.3]
geo_json_data = json.load(open(data_folder +'/ch-cantons.topojson.json'))
# Stamen Terrain, Stamen Toner, Mapbox Bright, and Mapbox Control Room, cartodbpositron
m_switzerland = folium.Map(ch_coord, tiles='Mapbox Bright', zoom_start=8)


folium.TopoJson(
    geo_json_data, 
    object_path='objects.cantons',
    style_function=style_function
).add_to(m_switzerland)

m_switzerland